<a href="https://colab.research.google.com/github/hariharanrl/DeepLearning/blob/master/SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q FastAPI
!pip install -q vaderSentiment
from fastapi import FastAPI
from pydantic import BaseModel
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


app = FastAPI()

vader_analyzer = SentimentIntensityAnalyzer()

class Review(Model):
    product_id: str
    review_id: str
    review_text: str
    expected_sentiment: str
    method: str = "textblob"

def analyze_sentiment_textblob(review: str):
    analysis = TextBlob(review)
    polarity = analysis.sentiment.polarity
    if polarity > 0:
        return "positive"
    elif polarity == 0:
        return "neutral"
    else:
        return "negative"

def analyze_sentiment_vader(review: str):
    score = vader_analyzer.polarity_scores(review)
    compound = score['compound']
    if compound >= 0.05:
        return "positive"
    elif compound <= -0.05:
        return "negative"
    else:
        return "neutral"

@app.post("/analyze/")
def analyze_review(review: Review):
    if review.method == "textblob":
        sentiment = analyze_sentiment_textblob(review.text)
    elif review.method == "vader":
        sentiment = analyze_sentiment_vader(review.text)
    else:
        return {"error": "Invalid method."}

    response = {
        "product_id": review.product_id,
        "review_id": review.review_id,
        "review_text": review.review_text,
        "expected_sentiment": review.expected_sentiment,
        "actual_sentiment": sentiment,
        "method": review.method
    }

    return response